In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

Using TensorFlow backend.


In [2]:
DATA_DIR = Path('.', 'data', 'final', 'public')
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [3]:
if 'presotere' in globals():
    del presotere

In [4]:
class presotere(TransformerMixin,BaseEstimator):
    
    def __init__(self,caso,geocode,scaler):
        self.lencoder_col=list([])
        self.object_cols=list([])
        self.number_cols = list([])
        self.lencoder_col=list([])
        self._caso=caso
        self.legeo = LabelEncoder()
        self.lencoder= list([])
        self._sns_data=None
        self._y=None
        self._geocode=geocode
        self._scaler=scaler
        
    def inicializamodelo(self,caso):

        if caso == 1:
            self.legeo = LabelEncoder()
            self.lencoder= list([])
        if caso == 2:
            self.legeo = LabelEncoder()
            self.lencoder= OneHotEncoder(handle_unknown='ignore', sparse=False)   
        
        
    def transform(self,X,y=None, **kwargs):
        print("Transformando ",self.get_params())
        if self._caso == 1:
            contador=0
            self._sns_data=X[self.number_cols].copy()
            for col in X[self.object_cols].columns:
                self.lencoder.append(LabelEncoder())
                self.lencoder[contador].fit(X[col])
                self._sns_data[col]=self.lencoder[contador].transform(X[col])  
                contador=contador+1
                
                
        if self._caso == 2:  
            nada = None
            self._sns_data=None
            nada = self.lencoder.fit_transform(X[self.object_cols])
            co1c=0
            self.lencoder_col=list([])
            for co1 in self.lencoder.categories_:
                for co2 in co1:
                    self.lencoder_col.append(self.object_cols[co1c]+"_"+co2)
                co1c=co1c+1
            objedf=pd.DataFrame(nada,columns=self.lencoder_col,
                                         index=X[self.object_cols].index.tolist())
            self._sns_data=pd.concat([X[self.number_cols].copy(),objedf],axis=1)  

                
        if self._geocode>0:        
            geo_level_1_fact=math.pow(10,int(math.log(self._sns_data['geo_level_2_id'].max(),10)+1))
            geo_level_2_fact=math.pow(10,int(math.log(self._sns_data['geo_level_3_id'].max(),10)+1))
            self._sns_data['geo_level_n']=  self._sns_data['geo_level_1_id']*geo_level_1_fact*geo_level_2_fact+self._sns_data['geo_level_2_id']*geo_level_2_fact+self._sns_data['geo_level_3_id']
            self._sns_data['geo_level']=self._sns_data['geo_level_n']#.astype(np.int64).astype(str)
            self._sns_data['geo_level_2']=self._sns_data['geo_level_n']*self._sns_data['geo_level_n']
            self._sns_data['geo_level_cod']=self._sns_data['geo_level_n'].astype(np.int64)

            self._sns_data=self._sns_data.drop(['geo_level_n','geo_level'],axis=1)

        self._sns_data=self._sns_data.drop(['count_floors_pre_eq'],axis=1)
        if self._scaler=="MinMax":
            mi_scaler = MinMaxScaler()
        if self._scaler=="Standard":
            mi_scaler = StandardScaler()            
        self._sns_data[list(self._sns_data.columns)]=mi_scaler.fit_transform(self._sns_data.values.astype(float))
        lcolum_x = list(self._sns_data.columns) 
        return self._sns_data
    
    def fit(self,X, y=None, **kwargs):
        for key, value in kwargs.items():
            if key=="caso":
                self._caso=value
            if key=="geocode":
                self._geocode=value
            if key=="scaler":
                self._scaler=value
        s = (X.dtypes == 'object')
        self.object_cols = list(s[s].index)
        s = (X.dtypes != 'object')        
        self.number_cols = list(s[s].index) 
        self._y=y
        self.inicializamodelo(self._caso)
        self.transform(X,y, **kwargs)
        return self

    def predict(self, X):
        return(self._y)

    def fit_transform(self,X, y=None, **kwargs):
            miX=X.values
            miX=miX.reshape(-1, 1)
            self.fit(X, y, **kwargs)
            nada=self.transform(X, **kwargs)
            return(nada)
        
    def set_params(self,**kwargs):
            for key, value in kwargs.items():
                if key=="caso":
                    self._caso=value
                if key=="geocode":
                    self._geocode=value
                if key=="scaler":
                    self._scaler=value                                 
            return self    
        
    def get_params(self,**kwargs):
        return({"caso":self._caso,"geocode":self._geocode,"scaler":self._scaler})
    

In [5]:
def cargamodelo(modelcaso):
    global model
    if modelcaso==1:
        model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=500)
    if modelcaso==2:    
        model = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=100)
    if modelcaso==3:
        model = DecisionTreeClassifier(random_state=0,max_depth=20) 
    if modelcaso==4:
        model = MultinomialNB
    if modelcaso==5:
        model = DecisionTreeRegressor(random_state=0)   
    if modelcaso==6:
        model = SVC(gamma='auto',verbose=True,kernel='linear', probability=True)       
    if modelcaso==7:
        model = RandomForestClassifier(max_depth=20, random_state=0,n_estimators=100)     
    if modelcaso==80:
        model = XGBClassifier()   
    if modelcaso==81:        
        model = XGBRegressor()        
    if modelcaso==9:    
        model = Sequential()
        model.add(Dense(12, input_dim=36, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(15, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      

In [6]:
class mimodelo(RandomForestRegressor):
    def predict(self, X):
        t = super().predict(X)
        return(t.round().astype("int64"))

In [7]:
if 'steps' in globals():
    del steps
if 'pipeline' in globals():    
    del pipeline
#steps = [('Preprosessor', presotere(caso=1,geocode=0,scaler="MinMax")),
#         ('Modelo',mimodelo(max_depth=5, random_state=0,n_estimators=10,verbose=2,n_jobs=2,objective='reg:squarederror'))]
steps = [('Preprosessor', presotere(caso=1,geocode=1,scaler="MinMax")),
         ('Modelo',mimodelo(max_depth=5, random_state=0,n_estimators=10,verbose=2,n_jobs=2))]
#steps = [('Preprosessor', presotere(caso=1))]
pipeline = Pipeline(steps)

In [24]:
pipeline.fit(train_values,train_labels,Preprosessor__caso=1,Preprosessor__geocode=1,Preprosessor__scaler="MinMax")
random_grid = {'Preprosessor__caso':[1],
               'Preprosessor__geocode':[1],
               'Preprosessor__scaler':["Standard"]
              }
print(random_grid)
grid = GridSearchCV(pipeline, param_grid=random_grid, cv=3,scoring="f1_micro", verbose=2,n_jobs=-1)
grid.fit(train_values,train_labels )
print("Best score",grid.best_score_)
print(grid.best_estimator_)

Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}
Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}


c:\program files\python37\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    3.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'Preprosessor__caso': [1], 'Preprosessor__geocode': [1], 'Preprosessor__scaler': ['Standard']}
Fitting 3 folds for each of 1 candidates, totalling 3 fits


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [40]:
pipeline.fit(train_values,train_labels)
nada=pipeline.predict(train_values)
##test_data=pipeline.predict(test_values)
#pvalues=test_values[['geo_level_1_id']]
#pvalues['damage_grade']=test_data.round().astype(np.int64)
#pvalues=pvalues.drop(['geo_level_1_id'],axis=1)
#pvalues.to_csv(DATA_DIR / 'submission_00_05.csv')


Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}
Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}


c:\program files\python37\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    3.3s finished


Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished


In [ ]:
#sns.scatterplot(train_values.index,nada)
nada1=pd.DataFrame(nada,columns=['result'])
nada1['llog']=np.log(nada1['result'])
sns.distplot(nada1, rug=True, hist=False)
#nada1.head()

In [45]:
pd.DataFrame(nada).to_csv(DATA_DIR / 'nada.csv')

SyntaxError: invalid syntax (<ipython-input-45-64d64f0421ad>, line 1)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'Preprosessor__caso':[1,2],
               'Preprosessor__geocode':[0,1],
               'Preprosessor__scaler':["MinMax","Standard"],
               'Modelo__n_estimators': n_estimators,
               'Modelo__max_features': max_features,
               'Modelo__max_depth': max_depth,
               'Modelo__min_samples_split': min_samples_split,
               'Modelo__min_samples_leaf': min_samples_leaf,
               'Modelo__bootstrap': bootstrap}
print(random_grid)

In [ ]:
grid = GridSearchCV(pipeline, param_grid=random_grid, cv=3,scoring="f1_micro", verbose=2,n_jobs=2)

In [ ]:
grid.fit(train_values,train_labels )

In [ ]:
print("Best score",grid.best_score_)
print(grid.best_estimator_)

In [8]:
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'Preprosessor__caso':[1,2],
               'Preprosessor__geocode':[0,1],
               'Preprosessor__scaler':["MinMax","Standard"],
               'Modelo__n_estimators':  sp_randint(200, 600),
               'Modelo__max_features': max_features,
               'Modelo__max_depth': sp_randint(10, 110),
               'Modelo__min_samples_split': min_samples_split,
               'Modelo__min_samples_leaf': min_samples_leaf,
               'Modelo__bootstrap': bootstrap}

In [9]:
grid = RandomizedSearchCV(pipeline,  param_distributions=random_grid, cv=3,scoring="f1_micro", verbose=2,n_jobs=2)

In [10]:
grid.fit(train_values,train_labels )

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed: 67.9min finished


Transformando  {'caso': 1, 'geocode': 0, 'scaler': 'Standard'}
Transformando  {'caso': 1, 'geocode': 0, 'scaler': 'Standard'}


c:\program files\python37\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 336building tree 2 of 336

building tree 3 of 336
building tree 4 of 336
building tree 5 of 336
building tree 6 of 336
building tree 7 of 336
building tree 8 of 336
building tree 9 of 336
building tree 10 of 336
building tree 11 of 336
building tree 12 of 336
building tree 13 of 336
building tree 14 of 336
building tree 15 of 336
building tree 16 of 336
building tree 17 of 336
building tree 18 of 336
building tree 19 of 336
building tree 20 of 336
building tree 21 of 336
building tree 22 of 336
building tree 23 of 336
building tree 24 of 336
building tree 25 of 336
building tree 26 of 336
building tree 27 of 336
building tree 28 of 336
building tree 29 of 336
building tree 30 of 336
building tree 31 of 336
building tree 32 of 336
building tree 33 of 336
building tree 34 of 336
building tree 35 of 336
building tree 36 of 336
building tree 37 of 336
building tree 38 of 336


...

building tree 39 of 336
building tree 40 of 336
building tree 41 of 336
building tree 42 of 336
building tree 43 of 336
building tree 44 of 336
building tree 45 of 336
building tree 46 of 336
building tree 47 of 336
building tree 48 of 336
building tree 49 of 336
building tree 50 of 336
building tree 51 of 336
building tree 52 of 336
building tree 53 of 336
building tree 54 of 336
building tree 55 of 336
building tree 56 of 336
building tree 57 of 336
building tree 58 of 336
building tree 59 of 336
building tree 60 of 336
building tree 61 of 336
building tree 62 of 336
building tree 63 of 336
building tree 64 of 336
building tree 65 of 336
building tree 66 of 336
building tree 67 of 336
building tree 68 of 336
building tree 69 of 336
building tree 70 of 336
building tree 71 of 336
building tree 72 of 336
building tree 73 of 336
building tree 74 of 336
building tree 75 of 336
building tree 76 of 336
building tree 77 of 336
building tree 78 of 336
building tree 79 of 336
building tree 80

[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:  2.3min


building tree 161 of 336
building tree 162 of 336
building tree 163 of 336
building tree 164 of 336
building tree 165 of 336
building tree 166 of 336
building tree 167 of 336
building tree 168 of 336
building tree 169 of 336
building tree 170 of 336
building tree 171 of 336
building tree 172 of 336
building tree 173 of 336
building tree 174 of 336
building tree 175 of 336
building tree 176 of 336
building tree 177 of 336
building tree 178 of 336
building tree 179 of 336
building tree 180 of 336
building tree 181 of 336
building tree 182 of 336
building tree 183 of 336
building tree 184 of 336
building tree 185 of 336
building tree 186 of 336
building tree 187 of 336
building tree 188 of 336
building tree 189 of 336
building tree 190 of 336
building tree 191 of 336
building tree 192 of 336
building tree 193 of 336
building tree 194 of 336
building tree 195 of 336
building tree 196 of 336
building tree 197 of 336
building tree 198 of 336
building tree 199 of 336
building tree 200 of 336


[Parallel(n_jobs=2)]: Done 336 out of 336 | elapsed:  4.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('Preprosessor', presotere(caso=1, geocode=1, scaler='MinMax')), ('Modelo', mimodelo(bootstrap=True, criterion='mse', max_depth=5, max_features='auto',
     max_leaf_nodes=None, min_impurity_decrease=0.0,
     min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
     oob_score=False, random_state=0, verbose=2, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=2,
          param_distributions={'Preprosessor__caso': [1, 2], 'Preprosessor__geocode': [0, 1], 'Preprosessor__scaler': ['MinMax', 'Standard'], 'Modelo__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000157B3400>, 'Modelo__max_features': ['auto', 'sqrt'], 'Modelo__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000157B3550>, 'Modelo__min_samples_split': [2, 5, 10

In [13]:
print("Best score",grid.best_score_)
print(grid.best_estimator_)

Best score 0.7343256549284155
Pipeline(memory=None,
     steps=[('Preprosessor', presotere(caso=1, geocode=0, scaler='Standard')), ('Modelo', mimodelo(bootstrap=True, criterion='mse', max_depth=27, max_features='auto',
     max_leaf_nodes=None, min_impurity_decrease=0.0,
     min_impurity_split=None, min_samples_leaf=2, min_samples_split=10,
     min_weight_fraction_leaf=0.0, n_estimators=336, n_jobs=2,
     oob_score=False, random_state=0, verbose=2, warm_start=False))])


In [14]:
scores_df = pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score')

c:\program files\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\program files\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\program files\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\program files\python37\lib\site-packages\sklearn\

In [15]:
scores_df

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0     235.127333      7.645898         6.436000        0.601981   
7      78.486667      1.487176         6.750000        0.401159   
1     395.281333     13.183942         7.351667        1.113255   
5      92.577333      5.986018         9.420000        0.387433   
8     100.334333      1.519398        10.214333        0.421945   
3     123.782667      2.410511        11.933667        0.440692   
2     121.124000      1.857561        10.812333        0.551317   
6      86.133333      1.384094         5.482667        0.301841   
9     519.631333     50.331512        11.845666        2.213254   
4     535.321333     19.132036        12.481667        0.717868   

  param_Modelo__bootstrap param_Modelo__max_depth param_Modelo__max_features  \
0                    True                      27                       auto   
7                    True                      55                       sqrt   
1                    True                      17                       auto   
5                    True                      81                       sqrt   
8                    True                      78                       sqrt   
3                   False                      78                       sqrt   
2                    True                      76                       sqrt   
6                   False                      20                       sqrt   
9                   False                     107                       auto   
4                   False                      96                       auto   

  param_Modelo__min_samples_leaf param_Modelo__min_samples_split  \
0                              2                              10   
7                              4                               5   
1                              4                               5   
5                              2                               5   
8                              2                               5   
3                              1                              10   
2                              2                               5   
6                              1                               5   
9                              4                               2   
4                              4                               5   

  param_Modelo__n_estimators  ... split1_test_score split2_test_score  \
0                        336  ...          0.735354          0.735561   
7                        389  ...          0.733409          0.733086   
1                        404  ...          0.727411          0.727204   
5                        428  ...          0.722576          0.721574   
8                        492  ...          0.722300          0.721379   
3                        358  ...          0.719168          0.719583   
2                        501  ...          0.711018          0.710581   
6                        387  ...          0.695776          0.694752   
9                        567  ...          0.683689          0.685796   
4                        525  ...          0.683585          0.685796   

  mean_test_score std_test_score  rank_test_score  split0_train_score  \
0        0.734326       0.001603                1            0.862266   
7        0.732933       0.000464                2            0.796770   
1        0.725803       0.002129                3            0.779738   
5        0.721302       0.001167                4            0.841689   
8        0.721152       0.001042                5            0.841505   
3        0.717952       0.002020                6            0.921196   
2        0.708512       0.003239                7            0.836094   
6        0.694986       0.000574                8            0.776728   
9        0.684157       0.001194                9            0.840037   
4        0.684099       0.001230               10            0.840014   

   split1_train_score  spli

In [18]:
final=Pipeline(memory=None,
     steps=[('Preprosessor', presotere(caso=1, geocode=1, scaler='Standard')), ('Modelo', mimodelo(bootstrap=True, criterion='mse', max_depth=27, max_features='auto',
     max_leaf_nodes=None, min_impurity_decrease=0.0,
     min_impurity_split=None, min_samples_leaf=2, min_samples_split=20,
     min_weight_fraction_leaf=0.0, n_estimators=336, n_jobs=2,
     oob_score=False, random_state=0, verbose=2, warm_start=False))])


In [19]:
final.fit(train_values,train_labels)
test_data=final.predict(test_values)
pvalues=test_values[['geo_level_1_id']].copy()
pvalues['damage_grade']=test_data
pvalues=pvalues.drop(['geo_level_1_id'],axis=1)
pvalues.to_csv(DATA_DIR / 'submission_NN_00_04.csv')

Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'Standard'}
Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'Standard'}


c:\program files\python37\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 336building tree 2 of 336

building tree 3 of 336
building tree 4 of 336
building tree 5 of 336
building tree 6 of 336
building tree 7 of 336
building tree 8 of 336
building tree 9 of 336
building tree 10 of 336
building tree 11 of 336
building tree 12 of 336
building tree 13 of 336
building tree 14 of 336
building tree 15 of 336
building tree 16 of 336
building tree 17 of 336
building tree 18 of 336
building tree 19 of 336
building tree 20 of 336
building tree 21 of 336
building tree 22 of 336
building tree 23 of 336
building tree 24 of 336
building tree 25 of 336
building tree 26 of 336
building tree 27 of 336
building tree 28 of 336
building tree 29 of 336
building tree 30 of 336
building tree 31 of 336
building tree 32 of 336
building tree 33 of 336
building tree 34 of 336
building tree 35 of 336
building tree 36 of 336
building tree 37 of 336
building tree 38 of 336
building tree 39 of 336
building tree 40 of 336


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   38.3s


building tree 41 of 336
building tree 42 of 336
building tree 43 of 336
building tree 44 of 336
building tree 45 of 336
building tree 46 of 336
building tree 47 of 336
building tree 48 of 336
building tree 49 of 336
building tree 50 of 336
building tree 51 of 336
building tree 52 of 336
building tree 53 of 336
building tree 54 of 336
building tree 55 of 336
building tree 56 of 336
building tree 57 of 336
building tree 58 of 336
building tree 59 of 336
building tree 60 of 336
building tree 61 of 336
building tree 62 of 336
building tree 63 of 336
building tree 64 of 336
building tree 65 of 336
building tree 66 of 336
building tree 67 of 336
building tree 68 of 336
building tree 69 of 336
building tree 70 of 336
building tree 71 of 336
building tree 72 of 336
building tree 73 of 336
building tree 74 of 336
building tree 75 of 336
building tree 76 of 336
building tree 77 of 336
building tree 78 of 336
building tree 79 of 336
building tree 80 of 336
building tree 81 of 336
building tree 82

[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:  2.7min


building tree 161 of 336
building tree 162 of 336
building tree 163 of 336
building tree 164 of 336
building tree 165 of 336
building tree 166 of 336
building tree 167 of 336
building tree 168 of 336
building tree 169 of 336
building tree 170 of 336
building tree 171 of 336
building tree 172 of 336
building tree 173 of 336
building tree 174 of 336
building tree 175 of 336
building tree 176 of 336
building tree 177 of 336
building tree 178 of 336
building tree 179 of 336building tree 180 of 336

building tree 181 of 336
building tree 182 of 336
building tree 183 of 336
building tree 184 of 336
building tree 185 of 336
building tree 186 of 336
building tree 187 of 336
building tree 188 of 336
building tree 189 of 336
building tree 190 of 336
building tree 191 of 336
building tree 192 of 336
building tree 193 of 336
building tree 194 of 336
building tree 195 of 336
building tree 196 of 336
building tree 197 of 336
building tree 198 of 336
building tree 199 of 336
building tree 200 of 336


[Parallel(n_jobs=2)]: Done 336 out of 336 | elapsed:  5.8min finished


Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'Standard'}


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    1.7s
[Parallel(n_jobs=2)]: Done 336 out of 336 | elapsed:    3.8s finished
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
